In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [60]:
!ls ../raw_data/

2016-17          2018-19          2020-21
2017-18          2019-20          full_data.pickle


## Import all datasets for the full years

In [61]:
# Importing each season as a dataframe and adding a column identifying the season
raw_data_path = '../raw_data'

df21 = pd.read_csv(raw_data_path + '/2020-21/gws/merged_gw.csv', encoding='utf_8')
df20 = pd.read_csv(raw_data_path + '/2019-20/gws/merged_gw.csv', encoding='utf_8')
df19 = pd.read_csv(raw_data_path + '/2018-19/gws/merged_gw.csv', encoding='ISO-8859-1')
df18 = pd.read_csv(raw_data_path + '/2017-18/gws/merged_gw.csv', encoding='ISO-8859-1')
df17 = pd.read_csv(raw_data_path + '/2016-17/gws/merged_gw.csv', encoding='ISO-8859-1')

df21['season'] = 21
df20['season'] = 20
df19['season'] = 19
df18['season'] = 18
df17['season'] = 17

In [62]:
dflist = [df17, df18, df19, df20, df21]

In [63]:
df19.loc[1799, ['name']]

name    Isaac_Mbenza_514
Name: 1799, dtype: object

In [64]:
df21.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,season
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,32.0,1,0,0,0,55,True,0,1,21
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,0.0,1,0,0,0,50,True,0,1,21
2,Aaron Mooy,MID,Brighton,0.0,0,0,0,0,0.0,60,...,0.0,0,0,0,0,50,True,0,1,21
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,0.0,1,0,0,0,50,True,0,1,21
4,Abdoulaye Doucouré,MID,Everton,1.3,0,0,20,1,44.6,512,...,4.0,3,0,0,0,55,False,0,1,21


In [65]:
df21.dtypes

name             object
position         object
team             object
xP              float64
assists           int64
                 ...   
value             int64
was_home           bool
yellow_cards      int64
GW                int64
season            int64
Length: 37, dtype: object

## Create list with all consistent column names throughout all years

In [66]:
consistent_columns = []
for column in list(df21.columns):
    if column in list(df20.columns) and \
       column in list(df19.columns) and \
       column in list(df18.columns) and \
       column in list(df17.columns):
        consistent_columns.append(column)

In [67]:
consistent_columns

['name',
 'assists',
 'bonus',
 'bps',
 'clean_sheets',
 'creativity',
 'element',
 'fixture',
 'goals_conceded',
 'goals_scored',
 'ict_index',
 'influence',
 'kickoff_time',
 'minutes',
 'opponent_team',
 'own_goals',
 'penalties_missed',
 'penalties_saved',
 'red_cards',
 'round',
 'saves',
 'selected',
 'team_a_score',
 'team_h_score',
 'threat',
 'total_points',
 'transfers_balance',
 'transfers_in',
 'transfers_out',
 'value',
 'was_home',
 'yellow_cards',
 'GW',
 'season']

## Check which columns are missing in other years

In [68]:
# Creating a dictionary with columns that are only in some years. 
# Key = column name, value = years for which the column exists.
missing_columns = {}
for df in dflist:
    for column in list(df.columns):
        if not column in consistent_columns:
            if column in missing_columns.keys():
                missing_columns[column].append(df.loc[0,['season']][0])
            else:
                missing_columns[column] = [df.loc[0,['season']][0]]
                
missing_columns

{'attempted_passes': [17, 18, 19],
 'big_chances_created': [17, 18, 19],
 'big_chances_missed': [17, 18, 19],
 'clearances_blocks_interceptions': [17, 18, 19],
 'completed_passes': [17, 18, 19],
 'dribbles': [17, 18, 19],
 'ea_index': [17, 18, 19],
 'errors_leading_to_goal': [17, 18, 19],
 'errors_leading_to_goal_attempt': [17, 18, 19],
 'fouls': [17, 18, 19],
 'id': [17, 18, 19],
 'key_passes': [17, 18, 19],
 'kickoff_time_formatted': [17, 18, 19],
 'loaned_in': [17, 18, 19],
 'loaned_out': [17, 18, 19],
 'offside': [17, 18, 19],
 'open_play_crosses': [17, 18, 19],
 'penalties_conceded': [17, 18, 19],
 'recoveries': [17, 18, 19],
 'tackled': [17, 18, 19],
 'tackles': [17, 18, 19],
 'target_missed': [17, 18, 19],
 'winning_goals': [17, 18, 19],
 'position': [21],
 'team': [21],
 'xP': [21]}

## Concatenate years based on consistent columns

In [69]:
# Concatenate seasons based on consistent columns
frames_to_concat = [df21[consistent_columns], 
                    df20[consistent_columns], 
                    df19[consistent_columns], 
                    df18[consistent_columns], 
                    df17[consistent_columns]]

complete_data = pd.concat(frames_to_concat)

In [70]:
# Drop column 'round' as it is the same as 'GW'
complete_data.drop(columns = ['round'], inplace = True)

In [80]:
complete_data.name = complete_data.name.str.rstrip('_1234567890')

In [88]:
complete_data.name = complete_data.name.str.replace('_', ' ')

In [100]:
complete_data.name.value_counts().head(20)

Ben Davies                 207
Sergio Agüero              191
Kevin De Bruyne            191
Ross Barkley               191
Cédric Soares              191
Nicolás Otamendi           191
John Stones                191
Raheem Sterling            191
Fernando Luiz Rosa         191
Calum Chambers             191
Kyle Walker                191
Ilkay Gündogan             191
Mesut Özil                 191
Héctor Bellerín            191
Granit Xhaka               191
Riyad Mahrez               191
Rob Holding                191
Oriol Romeu Vidal          190
Alex Oxlade-Chamberlain    190
Tom Davies                 190
Name: name, dtype: int64

In [90]:
complete_data.name.nunique()

1517

In [50]:
#complete_data.to_pickle('../raw_data/full_data.pickle')

In [51]:
#test = pd.read_pickle('../raw_data/full_data.pickle')

In [53]:
#test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114861 entries, 0 to 23678
Data columns (total 33 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   name               114861 non-null  object 
 1   assists            114861 non-null  int64  
 2   bonus              114861 non-null  int64  
 3   bps                114861 non-null  int64  
 4   clean_sheets       114861 non-null  int64  
 5   creativity         114861 non-null  float64
 6   element            114861 non-null  int64  
 7   fixture            114861 non-null  int64  
 8   goals_conceded     114861 non-null  int64  
 9   goals_scored       114861 non-null  int64  
 10  ict_index          114861 non-null  float64
 11  influence          114861 non-null  float64
 12  kickoff_time       114861 non-null  object 
 13  minutes            114861 non-null  int64  
 14  opponent_team      114861 non-null  int64  
 15  own_goals          114861 non-null  int64  
 16  pen